# Imports

In [56]:
import re
import numpy as np
from collections import Counter
import pandas as pd

# Code to read data

#### Function to read required data from txt file

In [49]:
def read_file(filename):
    with open(filename, 'r') as file:
        content = file.read()  # Open the file in read mode

    data = {}  # Initialize data as an empty dictionary

    # Each file has number of inputs under which data is stored as a matrix
    numerical_data = re.findall(r'\[([\d\s.]+)\]', content)  # Extract values following each header as numerical values and remove '\t's and '\n's

    sections = ['Orders', 'Allocations', 'DistanceShelfShelf', 'DistancePackagingShelf', 'FullDistanceMatrixMetres']

    for i, section in enumerate(sections):
        data[section] = []
        lines = numerical_data[i].strip().split('\n')
        for line in lines:
            values = line.strip().split()  # Split using spaces
            data[section].append([int(val) for val in values])  # Assuming all values are integers

    return data

#### Assigning the data we read to variables

In [50]:
filename = r"Xpress_Data_Files/Data_Xpress_FullDist_Metres.txt"
data = read_file(filename)

Orders = data.get('Orders')
Allocations = np.asarray(data.get('Allocations')[0])
DistanceShelfShelf = np.asarray(data.get('DistanceShelfShelf'))
FullDistanceMatrix = np.asarray(data.get('FullDistanceMatrixMetres'))

NbShelves = 96
Shelves = range(1, NbShelves + 1)

FullDistanceMatrix = np.roll(FullDistanceMatrix, shift = 1, axis = 1)
FullDistanceMatrix = np.roll(FullDistanceMatrix, shift = 1, axis = 0)

# Functions

In [51]:
def getMostCommonProducts(Orders):
    '''
    This function takes the order matrix and extracts a list of all 
    product groups, ordered by the number of occurrences in the matrix.

    This lets us know which products are the most commonly ordered.
    '''
    flattenedList = [x for x in [item for sublist in Orders for item in sublist] if x != 0]

    itemCounts = Counter(flattenedList)

    sorted_items = sorted(itemCounts.items(), key=lambda x: x[1], reverse=True)
    original_numbers = [item for item, _ in sorted_items]

    return itemCounts, original_numbers


def getClosestShelves(Distances):
    ''' 
    This function takes the distance vector and extracts a list of 
    shelves, ordered by the ones that are closest to the door (shelf 0 in our case).

    Returns a list of tuples containing the shelf index and its corresponding distance.
    '''
    distanceToDoor = Distances[0] # consider the distance vector for the door
    indexed_numbers = list(enumerate(distanceToDoor)) # indices and distances from door

    sorted_indexed_numbers = sorted(indexed_numbers, key=lambda x: x[1]) # sort indices by distance from door (ascending)
    sorted_indexes = [tuple([index,dist])  for index, dist in sorted_indexed_numbers if index > 0]

    return sorted_indexes # print index of shelf and its corresponding distance


def ConstructionHeuristic_mostCommonOrder(Orders, Distances, fill_extra_shelves = True, nbAdditionalShelves=6):
    '''
    This function first finds the most common product groups, then 
    extracts the list of shelves closest to the door. 

    It uses these two lists in conjunction to assign the 
    most common products to the closest shelves.

    If the 6 extra shelves are required to be filled, the function 
    will assign the most common product to the extra shelves.
    '''

    mostCommonProducts = getMostCommonProducts(Orders)[1] # get the most common products

    # if statement to decide whether you want to fill extra shelves or not
    # nbAdditionalShelves determines how many extra shelves you fill
    if fill_extra_shelves == True:
        mostCommonProducts = mostCommonProducts + mostCommonProducts[:nbAdditionalShelves] 
    elif fill_extra_shelves == False:
        mostCommonProducts = mostCommonProducts

    ClosestShelves, ClosestDistances = map(list, zip(*getClosestShelves(Distances)))

    newShelves = [0]*96  # initialise empty shelf allocation vector

    for i, prod in zip(ClosestShelves, mostCommonProducts):
        newShelves[i-1] = prod  # assign products based on the closest shelves

    return np.array(newShelves)


def Random_Allocation_heuristic(fill_extra_shelves=True, nbAdditionalShelves=6, seed=123):
    '''
    Construction heuristic to assign products to shelves randomly.

    Can provide additional arguments to randomly assign products to the extra shelves as well.
    '''
    np.random.seed(seed)
    prods_on_shelves = np.zeros(96, dtype = int)
    first_90_shelves = np.arange(1, 91) # assign the 90 products for the first time to the first 90 shelves
    np.random.shuffle(first_90_shelves) # randomly shuffle the products among the 90 shelves
    prods_on_shelves[:90] = first_90_shelves
    if fill_extra_shelves: # if we wish to fill some of the remaining empty shelves
        
        # randomly select nbAdditionalShelves number of products from the list of products and assign these to the empty shelves
        prods_on_shelves[90:90+nbAdditionalShelves] = np.random.choice(np.arange(1, 91), size = nbAdditionalShelves, replace=False)

    return prods_on_shelves

# Writing allocation vectors to txt files

#### Function to take an allocation vector and write it to a txt file

In [52]:
def allocation_to_txt(allocation_vector, file_path):
    np.savetxt(file_path, allocation_vector, fmt = "%d", delimiter = ",", newline=" ")

#### Writing current allocation to txt file

In [53]:
mostCommonOrderAllocation = ConstructionHeuristic_mostCommonOrder(Orders, 
                                                                  FullDistanceMatrix, 
                                                                  fill_extra_shelves=False)
allocation_to_txt(mostCommonOrderAllocation, "Allocation_Vectors/CommonOrder_0_Extra.txt")
allocation_to_txt(Random_Allocation_heuristic(nbAdditionalShelves=0), "Allocation_Vectors/Random_0_Extra.txt")

#### Writing allocation vectors with additional shelves filled to txt

In [54]:
for i in range(6):
    allocation_extra_shelves = ConstructionHeuristic_mostCommonOrder(Orders,
                                                                     FullDistanceMatrix,
                                                                     fill_extra_shelves=True,
                                                                     nbAdditionalShelves=i+1)
    allocation_to_txt(allocation_extra_shelves, f"Allocation_Vectors/CommonOrder_{i+1}_Extra.txt")
    allocation_to_txt(Random_Allocation_heuristic(nbAdditionalShelves=i+1), f"Allocation_Vectors/Random_{i+1}_Extra.txt")

#### Using our own distance matrices to construct allocation vectors and writing to txt 

In [57]:
for j in range(1,5):
    distance_matrix = pd.read_excel(f"Distance_Matrices/DistanceMatrix_{j}_aisles.xlsx", 
                                    sheet_name = "DistanceMatrixMetres", 
                                    header=None, skiprows= 1)
    allocation_0_extra = ConstructionHeuristic_mostCommonOrder(Orders, 
                                                                  distance_matrix, 
                                                                  fill_extra_shelves=False)
    allocation_6_extra = ConstructionHeuristic_mostCommonOrder(Orders, 
                                                                  distance_matrix)
    allocation_to_txt(allocation_0_extra, f"Allocation_Vectors/CommonOrder_0extra_{j}mid_aisle.txt")
    allocation_to_txt(allocation_6_extra, f"Allocation_Vectors/CommonOrder_6extra_{j}mid_aisle.txt")